## 读取markdown文件中理论假设的内容，提取关键词，以方便后续赋值

In [1]:
import pandas as pd
import re
import os
import numpy as np
from IPython.display import display

In [2]:
# Asin_List_file = "旅行包-机会款式.xlsx" 

# # 文件和目录的设置
# input_file = Asin_List_file  # 这实际上是Excel文件

# # 从Excel文件名中提取基础文件夹名称
# folder_name = os.path.splitext(input_file)[0]

folder_name = '生成结果/social_media/'
print(f"提取的文件夹名: {folder_name}")

# 创建文件夹（如果不存在）
# if not os.path.exists(folder_name):
#     os.makedirs(folder_name)
#     print(f"已创建文件夹: {folder_name}")


提取的文件夹名: 生成结果/social_media/


In [3]:
# Asin_List_file = "旅行包-机会款式.xlsx" 

# # 文件和目录的设置
# input_file = Asin_List_file  # 这实际上是Excel文件

# # 从Excel文件名中提取基础文件夹名称
# folder_name = os.path.splitext(input_file)[0]

folder_name = '生成结果/social_media/'
print(f"提取的文件夹名: {folder_name}")

# 创建文件夹（如果不存在）
# if not os.path.exists(folder_name):
#     os.makedirs(folder_name)
#     print(f"已创建文件夹: {folder_name}")


提取的文件夹名: 生成结果/social_media/


In [4]:
def extract_markdown_table(markdown_file):
    """
    从Markdown文件中提取表格内容并转换为DataFrame
    
    参数:
    markdown_file - Markdown文件路径
    
    返回:
    pandas DataFrame - 包含表格内容的数据框
    """
    # 读取Markdown文件内容
    with open(markdown_file, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # 按行分割内容
    lines = content.strip().split('\n')
    
    # 找到表格行（以|开始和结束的行）
    table_lines = []
    in_table = False
    
    for line in lines:
        line = line.strip()
        if line.startswith('|') and line.endswith('|'):
            if not in_table:
                in_table = True
                table_lines = []
            table_lines.append(line)
        elif in_table and not line:
            # 表格结束，找到了一个完整表格
            break
    
    if not table_lines:
        raise ValueError("未在Markdown文件中找到表格内容")
    
    # 解析表格内容
    rows = []
    for line in table_lines:
        # 分割行并移除前后的|
        cells = line.strip().split('|')[1:-1]
        # 清理每个单元格内容
        cells = [cell.strip() for cell in cells]
        rows.append(cells)
    
    # 第一行是表头
    headers = rows[0]
    
    # 跳过表头和分隔行(通常是第二行，包含 ---|--- 类似内容)
    data_rows = rows[2:] if len(rows) > 2 else []
    
    # 创建DataFrame
    df = pd.DataFrame(data_rows, columns=headers)
    
    print(f"已成功从Markdown提取表格，包含{len(df)}行和{len(df.columns)}列")
    
    return df

# 使用示例
# 假设Markdown文件路径
markdown_file = os.path.join(folder_name, '理论假设.md')

# 检查文件是否存在
if os.path.exists(markdown_file):
    # 从Markdown文件提取表格
    variables_df = extract_markdown_table(markdown_file)
    
    # 显示提取的DataFrame
    print("\n提取的DataFrame头部:")
    display(variables_df.head())
else:
    print(f"文件 {markdown_file} 不存在，请提供正确的文件路径")

已成功从Markdown提取表格，包含30行和10列

提取的DataFrame头部:


,序号,变量名称,数据字段描述,关键词列表,数据平台（待定）,理论定义,变量所在假设,调节/背景因素,变量类型,消费者旅程阶段
0,1,安全性需求表达,内容中包含对鞋架安全性的关注表达,"safety, stable, sturdy, secure, tipping, wobbl...","Twitter, Reddit, Instagram, Facebook",消费者对产品稳定性、防倾倒和整体安全性能的关注程度,"H1, H5",家庭结构、有无儿童/宠物,IV,认知/意识
1,2,互动量-安全性相关,"安全性相关内容的interaction_like_count, interaction_co...",与安全性需求表达关键词相关的互动指标,"Twitter, Reddit, Instagram, Facebook",消费者对安全性相关内容的互动响应程度,"H1, H5",家庭结构、居住环境,DV,兴趣/考虑
2,3,空间效率表达,内容中对鞋架空间节省、容量的讨论,"space-saving, compact, small space, apartment,...","Twitter, Reddit, Instagram, Facebook",消费者对鞋架空间利用效率的重视程度,"H2, H23",居住空间大小、住房类型,IV,认知/意识
3,4,分享行为-空间效率,"空间效率相关内容的interaction_share_count, interaction_...",与空间效率表达关键词相关的分享指标,"Twitter, Reddit, Instagram, Facebook",消费者分享空间效率相关内容的频率和强度,"H2, H23",居住空间大小、城市化程度,DV,倡导/推荐
4,5,环保材质偏好,内容中对环保材质的讨论和偏好,"eco-friendly, sustainable, recycled, natural, ...","Twitter, Reddit, Instagram, Facebook",消费者对环保材质的关注程度和偏好强度,H3,消费者环保价值观,IV,评估/比较


## 提取关键词列表

In [5]:
# 创建变量信息字典和变量名称列表
variable_keywords = {}
variable_names = []  # 存储所有变量名称的列表

# 从DataFrame中提取变量名称和对应的关键词列表
for _, row in variables_df.iterrows():
    var_name = row['变量名称']
    keyword_text = row['关键词列表']
    
    # 跳过非法行
    if not isinstance(keyword_text, str):
        continue
    
    # 按逗号分割关键词并去除空格
    keywords = [kw.strip() for kw in keyword_text.split(',')]
    
    # 保存到字典和列表
    if keywords:
        variable_keywords[var_name] = keywords
        variable_names.append(var_name)  # 添加到变量名称列表

# 显示提取的关键词
print("提取的变量关键词:")
for var_name, keywords in variable_keywords.items():
    print(f"{var_name}: {keywords}")

print(f"\n共提取了 {len(variable_names)} 个变量名称")

提取的变量关键词:
安全性需求表达: ['safety', 'stable', 'sturdy', 'secure', 'tipping', 'wobble', 'fall']
互动量-安全性相关: ['与安全性需求表达关键词相关的互动指标']
空间效率表达: ['space-saving', 'compact', 'small space', 'apartment', 'organize', 'clutter']
分享行为-空间效率: ['与空间效率表达关键词相关的分享指标']
环保材质偏好: ['eco-friendly', 'sustainable', 'recycled', 'natural', 'environmentally', 'bamboo']
环保价值观表达: ['green living', 'sustainability', 'eco-conscious', 'planet', 'environment']
组装便捷性表达: ['easy assembly', 'instructions', 'setup', 'put together', 'tools', 'DIY']
正面评价-组装体验: ['与组装便捷性表达关键词相关的正面情感评价']
设计美感表达: ['stylish', 'modern', 'sleek', 'beautiful', 'design', 'aesthetic', 'look', 'decor']
分享行为-设计美感: ['与设计美感表达关键词相关的分享指标']
层高可调节性表达: ['adjustable', 'flexible', 'customizable', 'height', 'shelves', 'layers']
互动参与度-可调节性: ['与层高可调节性表达关键词相关的互动指标综合']
防尘功能表达: ['dust', 'cover', 'protection', 'clean', 'dirt', 'dustproof']
互动量-有儿童家庭: ['与防尘功能表达关键词相关的互动指标(来自有儿童家庭用户)']
价格表达: ['price', 'cost', 'expensive', 'cheap', 'affordable', 'value', 'worth', 'budget']
家庭收入水平表达: 

In [6]:
# 加载数据文件
import pandas as pd
import re
import unicodedata
from tqdm import tqdm
import spacy
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

## 使用预训练模型来进行赋值，为0,1二分变量

In [7]:
# 加载关键词筛选后的数据
data_file = os.path.join(folder_name, "关键词筛选后的数据.xlsx")

# 检查文件是否存在
if not os.path.exists(data_file):
    print(f"错误：找不到文件 {data_file}")
    print("请确保先运行数据筛选代码，生成关键词筛选后的数据文件")
    raise FileNotFoundError(f"找不到文件: {data_file}")

df = pd.read_excel(data_file)
print(f"数据加载完成，共 {len(df)} 行")

# 定义文本预处理函数
def preprocess_text(text):
    """对文本进行预处理"""
    if not isinstance(text, str):
        return ""
    
    # 转为小写
    text = text.lower()
    
    # 规范化为 NFKD 形式，处理复合字符
    text = unicodedata.normalize('NFKD', text)
    
    # 移除非ASCII字符（包括表情符号）
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    
    # 将多个空格替换为单个空格
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

# 初始化结果DataFrame
result_df = df.copy()

# 选择适合英文文本的预训练模型
#model_name = "sentence-transformers/all-MiniLM-L6-v2"  # 小型语义模型，适合英文文本

#model_name = "sentence-transformers/stsb-roberta-large"

model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

# 加载模型和分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 自动检测并使用最佳的加速设备（CUDA、MPS或CPU）
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("使用CUDA加速")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device("mps")
    print("使用MPS加速 (Apple Silicon)")
else:
    device = torch.device("cpu")
    print("使用CPU (没有可用的硬件加速)")

model.to(device)

# 获取文本嵌入向量的函数
def get_embedding(text, tokenizer, model):
    """使用Transformer模型获取文本的嵌入向量"""
    if not text:
        return None
    
    # 预处理文本
    text = preprocess_text(text)
    if not text:
        return None
    
    # 对文本进行编码
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # 获取模型输出
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 使用最后一层的隐藏状态的平均值作为句子嵌入
    embeddings = outputs.last_hidden_state.mean(dim=1)
    
    # 将结果转移到CPU以便进行NumPy转换
    return embeddings.cpu().numpy()[0]

# 计算相似度分数并二元化
def semantic_match(text_embedding, keyword_embeddings, threshold=0.4):  # 降低默认阈值
    """计算文本与关键词的语义相似度，并基于阈值返回0或1"""
    if text_embedding is None or not keyword_embeddings:
        return 0
    
    # 计算文本与每个关键词之间的余弦相似度
    similarities = [
        cosine_similarity(text_embedding.reshape(1, -1), kw_embedding.reshape(1, -1))[0][0]
        for kw_embedding in keyword_embeddings if kw_embedding is not None
    ]
    
    # 如果没有有效的相似度分数，返回0
    if not similarities:
        return 0
    
    # 2. 使用多种匹配策略综合考虑
    max_similarity = max(similarities)
    avg_similarity = sum(similarities) / len(similarities)
    
    # 综合考虑最大相似度和平均相似度
    combined_score = 0.7 * max_similarity + 0.3 * avg_similarity
    
    # 基于阈值二元化
    return 1 if combined_score >= threshold else 0

# 预处理关键词，获取每个变量关键词的嵌入向量
print("计算关键词嵌入向量...")
keyword_embeddings = {}

for var_name, keywords in tqdm(variable_keywords.items()):
    keyword_embeddings[var_name] = []
    for keyword in keywords:
        embedding = get_embedding(keyword, tokenizer, model)
        keyword_embeddings[var_name].append(embedding)

# 预处理并获取所有内容的嵌入向量
print("计算内容嵌入向量...")
content_embeddings = []

if 'content' in df.columns:
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        content = row['content']
        embedding = get_embedding(content, tokenizer, model)
        content_embeddings.append(embedding)
else:
    print("警告: 数据中没有'content'列")
    content_embeddings = [None] * len(df)

# 为每个变量计算语义匹配得分
print("计算语义匹配得分...")
for var_name in tqdm(variable_names):
    print(f"处理变量: {var_name}")
    
    # 获取该变量的关键词嵌入向量
    var_keyword_embeddings = keyword_embeddings.get(var_name, [])
    
    if not var_keyword_embeddings:
        print(f"警告: 变量 {var_name} 没有有效的关键词嵌入向量，跳过")
        result_df[var_name] = 0
        continue
    
    # 计算每个内容与该变量关键词的语义匹配分数（不二元化）
    raw_scores = []
    for content_embedding in content_embeddings:
        if content_embedding is None:
            raw_scores.append(0)
            continue
            
        # 计算相似度但不二元化
        similarities = [
            cosine_similarity(content_embedding.reshape(1, -1), kw_embedding.reshape(1, -1))[0][0]
            for kw_embedding in var_keyword_embeddings if kw_embedding is not None
        ]
        
        if not similarities:
            raw_scores.append(0)
            continue
            
        # 计算组合分数
        max_sim = max(similarities)
        avg_sim = sum(similarities) / len(similarities)
        combined_score = 0.7 * max_sim + 0.3 * avg_sim
        raw_scores.append(combined_score)
    
    # 4. 使用分位数设置动态阈值，确保有合理的0/1分布
    # 根据数据特性调整百分位数，这里选择前75%作为1
    if len(set(raw_scores)) > 1:  # 确保有足够的变化
        threshold = np.percentile(raw_scores, 60)  # 取第70百分位作为阈值
        binary_scores = [1 if score >= threshold else 0 for score in raw_scores]
    else:
        # 如果所有分数都相同，则全部标记为0
        binary_scores = [0] * len(raw_scores)
        
    # 保存到结果DataFrame
    result_df[var_name] = binary_scores
    
    # 5. 输出每个变量的分布情况
    ones_count = sum(binary_scores)
    zeros_count = len(binary_scores) - ones_count
    print(f"  变量 {var_name} 分布: 1={ones_count}个 ({ones_count/len(binary_scores)*100:.1f}%), 0={zeros_count}个")

# 显示结果统计
print("\n变量得分统计:")
for var_name in variable_names:
    if var_name in result_df.columns:
        # 计算二元变量的频率和百分比
        value_counts = result_df[var_name].value_counts()
        total = len(result_df)
        
        print(f"{var_name}:")
        for value, count in value_counts.items():
            percent = count / total * 100
            print(f"  值={value}: {count}个样本 ({percent:.2f}%)")

# 修改输出文件路径，将所有输出保存到该文件夹
output_file = os.path.join(folder_name, "数据变量语义匹配二元赋值结果.xlsx")
result_df.to_excel(output_file, index=False)
print(f"\n处理完成！结果已保存至: {output_file}")


# 展示结果DataFrame的前几行
print(result_df.head())

数据加载完成，共 605 行


使用CUDA加速


计算关键词嵌入向量...


  0%|          | 0/30 [00:00<?, ?it/s]

  3%|▎         | 1/30 [00:00<00:05,  5.50it/s]

...

  变量 儿童友好设计表达 分布: 1=245个 (40.5%), 0=360个
处理变量: 组合灵活性表达


100%|██████████| 30/30 [00:27<00:00,  1.12s/it]

100%|██████████| 30/30 [00:27<00:00,  1.09it/s]

  变量 组合灵活性表达 分布: 1=244个 (40.3%), 0=361个

变量得分统计:
安全性需求表达:
  值=0: 363个样本 (60.00%)
  值=1: 242个样本 (40.00%)
互动量-安全性相关:
  值=0: 605个样本 (100.00%)
空间效率表达:
  值=0: 363个样本 (60.00%)
  值=1: 242个样本 (40.00%)
分享行为-空间效率:
  值=0: 605个样本 (100.00%)
环保材质偏好:
  值=0: 363个样本 (60.00%)
  值=1: 242个样本 (40.00%)
环保价值观表达:
  值=0: 363个样本 (60.00%)
  值=1: 242个样本 (40.00%)
组装便捷性表达:
  值=0: 363个样本 (60.00%)
  值=1: 242个样本 (40.00%)
正面评价-组装体验:
  值=0: 605个样本 (100.00%)
设计美感表达:
  值=0: 363个样本 (60.00%)
  值=1: 242个样本 (40.00%)
分享行为-设计美感:
  值=0: 605个样本 (100.00%)
层高可调节性表达:
  值=0: 362个样本 (59.83%)
  值=1: 243个样本 (40.17%)
互动参与度-可调节性:
  值=0: 605个样本 (100.00%)
防尘功能表达:
  值=0: 363个样本 (60.00%)
  值=1: 242个样本 (40.00%)
互动量-有儿童家庭:
  值=0: 363个样本 (60.00%)
  值=1: 242个样本 (40.00%)
价格表达:
  值=0: 363个样本 (60.00%)
  值=1: 242个样本 (40.00%)
家庭收入水平表达:
  值=0: 362个样本 (59.83%)
  值=1: 243个样本 (40.17%)
多功能性表达:
  值=0: 363个样本 (60.00%)
  值=1: 242个样本 (40.00%)
评论互动-多功能性:
  值=0: 605个样本 (100.00%)
便携性表达:
  值=0: 363个样本 (60.00%)
  值=1: 242个样本 (40.00%)
居住流动性表达:
  值=0: 362个样本 (59.83%)


处理完成！结果已保存至: 生成结果/social_media/数据变量语义匹配二元赋值结果.xlsx
                    id  title  \
0  1909480340126580992    NaN   
1  1909020699747886080    NaN   
2  1908203431765881088    NaN   
3  1908194727209385984    NaN   
4  1908053656731087104    NaN   

                                             content  url lang  \
0  Metal Shoe Rack\nUpto 24% off\nLink for you: h...  NaN   en   
1  @emob_ @DamnNearWhite @Tr3sMilagros @MyFirstKi...  NaN   en   
2  J.T. Foote Adjustable Shoe Trees - Plastic \n🛠...  NaN   en   
3  EXVITO Metal Shoe Rack for Home – Adjustable &...  NaN   en   
4  Closet Shoe Organizer for 24 Pairs with Adjust...  NaN   en   

                     publish_time publish_time_date publish_time_time  \
0  Tue Apr 08 05:36:04 +0000 2025        2025-04-08          05:36:04   
1  Sun Apr 06 23:09:37 +0000 2025        2025-04-06          23:09:37   
2  Fri Apr 04 17:02:05 +0000 2025        2025-04-04          17:02:05   
3  Fri Apr 04 16:27:30 +0000 2025        2025-04-04         

In [8]:
# 方法1: 检查是否存在NaN值
null_counts = result_df.isnull().sum()
print("每列的NaN值数量:")
print(null_counts)

if null_counts.sum() > 0:
    print("警告: 数据中存在NaN值")
else:
    print("所有单元格都有值")

# 方法2: 检查所有变量列的唯一值分布
print("\n每个变量列的值分布:")
for var_name in variable_names:
    if var_name in result_df.columns:
        value_counts = result_df[var_name].value_counts(dropna=False)
        print(f"\n{var_name}:")
        print(value_counts)
        
        # 检查是否只有0值(可能表示赋值失败)
        if value_counts.get(0, 0) == len(result_df) and 1 not in value_counts:
            print(f"⚠️ 警告: 变量 {var_name} 可能未成功赋值，全部为0")

# 方法3: 检查数据类型，确保所有变量列都是数值类型
print("\n各列的数据类型:")
print(result_df.dtypes)

每列的NaN值数量:
id            0
title       605
content       0
url         605
lang          0
           ... 
品牌声誉表达        0
消费者信任度表达      0
创新功能表达        0
儿童友好设计表达      0
组合灵活性表达       0
Length: 67, dtype: int64
警告: 数据中存在NaN值

每个变量列的值分布:

安全性需求表达:
安全性需求表达
0    363
1    242
Name: count, dtype: int64

互动量-安全性相关:
互动量-安全性相关
0    605
Name: count, dtype: int64
⚠️ 警告: 变量 互动量-安全性相关 可能未成功赋值，全部为0

空间效率表达:
空间效率表达
0    363
1    242
Name: count, dtype: int64

分享行为-空间效率:
分享行为-空间效率
0    605
Name: count, dtype: int64
⚠️ 警告: 变量 分享行为-空间效率 可能未成功赋值，全部为0

环保材质偏好:
环保材质偏好
0    363
1    242
Name: count, dtype: int64

环保价值观表达:
环保价值观表达
0    363
1    242
Name: count, dtype: int64

组装便捷性表达:
组装便捷性表达
0    363
1    242
Name: count, dtype: int64

正面评价-组装体验:
正面评价-组装体验
0    605
Name: count, dtype: int64
⚠️ 警告: 变量 正面评价-组装体验 可能未成功赋值，全部为0

设计美感表达:
设计美感表达
0    363
1    242
Name: count, dtype: int64

分享行为-设计美感:
分享行为-设计美感
0    605
Name: count, dtype: int64
⚠️ 警告: 变量 分享行为-设计美感 可能未成功赋值，全部为0

层高可调节性表达:
层高可调节性表达
0    362
1    24

## 输出变量为txt文件

In [9]:
# 从Markdown表格中提取IV、DV和Moderator变量
import pandas as pd
import os


# 假设variables_df已经通过之前的代码从Markdown文件中提取
# 如果没有，可以重新运行提取代码
if 'variables_df' not in locals():
    # 重新从Markdown文件中提取表格
    markdown_file = "理论假设.md"
    
    # 检查文件是否存在
    if os.path.exists(markdown_file):
        with open(markdown_file, 'r', encoding='utf-8') as file:
            content = file.read()
        
        # 按行分割内容
        lines = content.strip().split('\n')
        
        # 找到表格行（以|开始和结束的行）
        table_lines = [line for line in lines if line.strip().startswith('|') and line.strip().endswith('|')]
        
        if not table_lines:
            raise ValueError("未在Markdown文件中找到表格内容")
        
        # 解析表格内容
        rows = []
        for line in table_lines:
            # 分割行并移除前后的|
            cells = line.strip().split('|')[1:-1]
            # 清理每个单元格内容
            cells = [cell.strip() for cell in cells]
            rows.append(cells)
        
        # 第一行是表头
        headers = rows[0]
        
        # 跳过表头和分隔行(通常是第二行，包含 ---|--- 类似内容)
        data_rows = rows[2:] if len(rows) > 2 else []
        
        # 创建DataFrame
        variables_df = pd.DataFrame(data_rows, columns=headers)
        
        print(f"已成功从Markdown提取表格，包含{len(variables_df)}行和{len(variables_df.columns)}列")
    else:
        print(f"文件 {markdown_file} 不存在，请提供正确的文件路径")
        raise FileNotFoundError(f"找不到文件: {markdown_file}")

# 查看DataFrame的列名，确保有'变量名称'和'变量类型'列
print("DataFrame的列名:")
print(variables_df.columns.tolist())

# 检查'变量类型'列是否存在
if '变量类型' not in variables_df.columns:
    # 尝试查找包含'类型'的列
    type_columns = [col for col in variables_df.columns if '类型' in col]
    if type_columns:
        var_type_col = type_columns[0]
        print(f"使用列 '{var_type_col}' 作为变量类型列")
    else:
        print("警告: DataFrame中没有找到'变量类型'或包含'类型'的列")
        raise ValueError("无法找到变量类型列")
else:
    var_type_col = '变量类型'

# 检查'变量名称'列是否存在
if '变量名称' not in variables_df.columns:
    # 尝试查找包含'名称'的列
    name_columns = [col for col in variables_df.columns if '名称' in col]
    if name_columns:
        var_name_col = name_columns[0]
        print(f"使用列 '{var_name_col}' 作为变量名称列")
    else:
        print("警告: DataFrame中没有找到'变量名称'或包含'名称'的列")
        raise ValueError("无法找到变量名称列")
else:
    var_name_col = '变量名称'

# 提取IV、DV和Moderator变量
iv_variables = variables_df[variables_df[var_type_col] == 'IV'][var_name_col].tolist()
dv_variables = variables_df[variables_df[var_type_col] == 'DV'][var_name_col].tolist()
moderator_variables = variables_df[variables_df[var_type_col] == 'Moderator'][var_name_col].tolist()

# 显示提取的变量
print(f"\n找到 {len(iv_variables)} 个IV变量:")
for var in iv_variables:
    print(f"- {var}")

print(f"\n找到 {len(dv_variables)} 个DV变量:")
for var in dv_variables:
    print(f"- {var}")

print(f"\n找到 {len(moderator_variables)} 个Moderator变量:")
for var in moderator_variables:
    print(f"- {var}")

# 修改IV、DV和Moderator文本文件的保存路径
iv_file = os.path.join(folder_name, 'IV.txt')
dv_file = os.path.join(folder_name, 'DV.txt')
moderator_file = os.path.join(folder_name, 'Moderator.txt')

# 保存到文本文件
# IV.txt
with open(iv_file, 'w', encoding='utf-8') as f:
    for var in iv_variables:
        f.write(f"{var}\n")

# DV.txt
with open(dv_file, 'w', encoding='utf-8') as f:
    for var in dv_variables:
        f.write(f"{var}\n")

# Moderator.txt
with open(moderator_file, 'w', encoding='utf-8') as f:
    for var in moderator_variables:
        f.write(f"{var}\n")

print("\n已成功保存以下文件:")
print(f"- {iv_file} ({len(iv_variables)} 个变量)")
print(f"- {dv_file} ({len(dv_variables)} 个变量)")
print(f"- {moderator_file} ({len(moderator_variables)} 个变量)")

DataFrame的列名:
['序号', '变量名称', '数据字段描述', '关键词列表', '数据平台（待定）', '理论定义', '变量所在假设', '调节/背景因素', '变量类型', '消费者旅程阶段']

找到 18 个IV变量:
- 安全性需求表达
- 空间效率表达
- 环保材质偏好
- 组装便捷性表达
- 设计美感表达
- 层高可调节性表达
- 防尘功能表达
- 价格表达
- 多功能性表达
- 便携性表达
- 承重能力表达
- 耐用性表达
- 颜色选择表达
- 季节适应性表达
- 品牌声誉表达
- 创新功能表达
- 儿童友好设计表达
- 组合灵活性表达

找到 9 个DV变量:
- 互动量-安全性相关
- 分享行为-空间效率
- 正面评价-组装体验
- 分享行为-设计美感
- 互动参与度-可调节性
- 互动量-有儿童家庭
- 评论互动-多功能性
- 长期满意度表达
- 消费者信任度表达

找到 0 个Moderator变量:

已成功保存以下文件:
- 生成结果/social_media/IV.txt (18 个变量)
- 生成结果/social_media/DV.txt (9 个变量)
- 生成结果/social_media/Moderator.txt (0 个变量)
